In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [3]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [4]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [5]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [6]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [7]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-1.7296e-01,  1.1119e-01,  3.3084e-01, -2.0164e-01,  1.3832e-01,
         -7.0031e-01,  1.5829e-01, -2.3841e-01,  2.0881e-01, -6.5979e-02,
         -1.8007e-01, -3.3127e-01,  3.0536e-01,  1.4653e-01, -4.2480e-01,
         -2.0498e-01, -2.3002e-01, -3.0111e-01,  6.5770e-01, -5.4459e-01],
        [-2.3032e-02,  2.4085e-02,  2.6553e-01,  7.8191e-02, -2.3790e-01,
         -7.0391e-01,  8.7228e-04,  1.7164e-01,  4.3313e-01, -2.3205e-02,
         -6.4675e-02,  3.5766e-01,  4.0510e-01,  1.1084e-01, -2.5826e-01,
         -2.4297e-01, -2.9458e-01, -2.0594e-01,  1.0095e+00, -5.1315e-01],
        [ 9.4626e-03,  1.6298e-01,  1.4243e-02,  2.7308e-02,  3.2181e-01,
         -8.1375e-01,  8.8218e-02,  1.9835e-01,  3.4593e-01, -1.0166e-01,
          1.6801e-01, -8.8549e-02,  4.4792e-01,  1.2947e-01, -4.7639e-01,
          4.9100e-02, -3.2299e-01, -1.7702e-02,  9.3972e-01, -3.4021e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000e+00, 1.1119e-01, 3.3084e-01, 0.0

In [8]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits.shape)

torch.Size([3, 10])


In [9]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0575, 0.1559, 0.1131, 0.0578, 0.0908, 0.1116, 0.1161, 0.0765, 0.1055,
         0.1151],
        [0.0606, 0.1401, 0.1111, 0.0571, 0.0861, 0.1212, 0.1195, 0.0706, 0.1077,
         0.1260],
        [0.0679, 0.1460, 0.1100, 0.0637, 0.0846, 0.1173, 0.1135, 0.0840, 0.1055,
         0.1075]], grad_fn=<SoftmaxBackward0>)


In [10]:
print(f"Model Structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0047, -0.0033,  0.0207,  ..., -0.0341, -0.0246,  0.0338],
        [ 0.0079, -0.0156,  0.0121,  ..., -0.0135,  0.0101, -0.0201]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0045,  0.0179], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0347,  0.0254,  0.0270,  ...,  0.0308,  0.0332, -0.0246],
        [ 0.0349,  0.0386,  0.0352,  ...,  0.0092, -0.0384,  0.0138]],
       device='cuda:0', grad_fn=<Slice